# Preparation

In [1]:
from aocd import get_data
from aocd import submit
import unittest

day = 12
year = 2023

def submit_part_a(answer):
    submit(answer, part="a", day=day, year=year)

def submit_part_b(answer):
    submit(answer, part="b", day=day, year=year)

input = get_data(day=day, year=year)

In [2]:
lines = input.split('\n')

# Part 1

## How to know an arrangement of `..#.##...###` is possible with groups `[1, 2, 3]`

In [3]:
def is_arrangement_possible(springs, groups):
    groups_present_in_spring = [len(s) for s in springs.split('.') if s != ""]
    return groups_present_in_spring == groups

assert is_arrangement_possible("....###", [1, 1, 3]) == False
assert is_arrangement_possible("##..###", [1, 1, 3]) == False
assert is_arrangement_possible("#.#.###", [1, 1, 3]) == True
assert is_arrangement_possible("##...##", [2, 2]) == True
assert is_arrangement_possible(".##..##", [2, 2]) == True
assert is_arrangement_possible("..##.##", [2, 2]) == True
assert is_arrangement_possible("##...##", [2, 2]) == True
assert is_arrangement_possible("##..##.", [2, 2]) == True
assert is_arrangement_possible("##.##..", [2, 2]) == True
assert is_arrangement_possible(".######", [6]) == True
assert is_arrangement_possible(".######", [5]) == False

## Counting possible arrangements for `??#.?#?????##` is possible with groups `[1, 2, 3]`

In [4]:
def count_arrangements(springs, groups):
    new_springs = ''
    if '?' in springs:
        return count_arrangements(springs.replace('?', '#', 1), groups) + count_arrangements(springs.replace('?', '.', 1), groups)
    if is_arrangement_possible(springs, groups):
        return 1
    else:
        return 0

assert count_arrangements("???.###", [1,1,3]) == 1
assert count_arrangements(".??..??...?##.", [1,1,3]) == 4
assert count_arrangements("?#?#?#?#?#?#?#?", [1,3,1,6]) == 1
assert count_arrangements("????.#...#...", [4,1,1]) == 1
assert count_arrangements("????.######..#####.", [1,6,5]) == 4
assert count_arrangements("?###????????", [3,2,1]) == 10

## Resoling the first part

In [5]:
%%time
first_answer = 0
for l in lines:
    springs, groups = l.split()
    groups = [int(group) for group in groups.split(",")]
    first_answer += count_arrangements(springs, groups)

print(first_answer)

6852
CPU times: user 15 s, sys: 5.88 ms, total: 15 s
Wall time: 15 s


In [6]:
submit_part_a(first_answer)

aocd will not submit that answer again. At 2023-12-12 02:18:25.653806-05:00 you've previously submitted 6852 and the server responded with:
That's the right answer!  You are one gold star closer to restoring snow operations. [Continue to Part Two]


# Part 2

## Avoiding brute force
Because the first part took 20 sec to be resolved, I have to find another way to count possible arrangements.

In [7]:
memoized = dict()
def memoized_count_arrangements(springs, groups):
    memoized_key = (springs, tuple(groups))
    if memoized_key not in memoized:
        memoized[memoized_key] = count_arrangements(springs, groups)
    return memoized[memoized_key]
    
def count_arrangements(springs, groups):
    count = 0
    # Break conditions of the recursive function
    # No more groups to put
    if not groups:
        # No more spring to complete with a group
        if '#' not in springs:
            # No more spring to complete with a group and no more groups remaining, it's a valid possibility, so return 1
            return 1
        # It remains a spring to complete but no more group to complete it, it's an invalid possibility, so return 0
        return 0
    
    # Not enough space to put the first group
    if len(springs) < groups[0]:
        # First group need {groups[0]} spaces but only {len(springs)} spaces remaining, it's an invalid possibility, so return 0
        return 0

    first_group = groups[0]
    # No '.' is in the first potential group, this means there is enough place to put the group here
    if "." not in springs[:first_group]:
        # The springs have the same length than the group, so it's a valid possibility, so keep counting
        # The first spring after the first group is not a '#', so it's a valid possibility, so keep counting
        if len(springs) == first_group or springs[first_group] != "#":
            # Count the rest like if we put the first_group at the begining => removing {first_group+1} first springs and the first group {groups[0]}
            count += memoized_count_arrangements(springs[first_group+1:], groups[1:])

    if springs[0] != "#":
        # Count the rest like if we choose to not put the first_group at the begining => removing first spring
        count += memoized_count_arrangements(springs[1:], groups)

    return count


assert count_arrangements("?", [1]) == 1
assert count_arrangements("??", [1]) == 2
assert count_arrangements("???", [1]) == 3
assert count_arrangements("???", [1, 1]) == 1
assert count_arrangements("?#?", [1]) == 1

assert count_arrangements("???.###", [1,1,3]) == 1
assert count_arrangements(".??..??...?##.", [1,1,3]) == 4
assert count_arrangements("?#?#?#?#?#?#?#?", [1,3,1,6]) == 1
assert count_arrangements("????.#...#...", [4,1,1]) == 1
assert count_arrangements("????.######..#####.", [1,6,5]) == 4
assert count_arrangements("?###????????", [3,2,1]) == 10

assert count_arrangements("???.###????.###????.###????.###????.###", [1,1,3,1,1,3,1,1,3,1,1,3,1,1,3]) == 1
assert count_arrangements(".??..??...?##.?.??..??...?##.?.??..??...?##.?.??..??...?##.?.??..??...?##.", [1,1,3,1,1,3,1,1,3,1,1,3,1,1,3]) == 16384
assert count_arrangements("?#?#?#?#?#?#?#???#?#?#?#?#?#?#???#?#?#?#?#?#?#???#?#?#?#?#?#?#???#?#?#?#?#?#?#?", [1,3,1,6,1,3,1,6,1,3,1,6,1,3,1,6,1,3,1,6]) == 1
assert count_arrangements("????.#...#...?????.#...#...?????.#...#...?????.#...#...?????.#...#...", [4,1,1,4,1,1,4,1,1,4,1,1,4,1,1]) == 16
assert count_arrangements("????.######..#####.?????.######..#####.?????.######..#####.?????.######..#####.?????.######..#####.", [1,6,5,1,6,5,1,6,5,1,6,5,1,6,5]) == 2500
assert count_arrangements("?###??????????###??????????###??????????###??????????###????????", [3,2,1,3,2,1,3,2,1,3,2,1,3,2,1]) == 506250

## Resoling the second part

In [8]:
%%time
second_answer = 0
for l in lines:
    springs, groups = l.split()
    groups = [int(group) for group in groups.split(",")]
    second_answer += count_arrangements((5 * (springs + '?'))[:-1], 5 * groups)

print(second_answer)

8475948826693
CPU times: user 802 ms, sys: 119 ms, total: 922 ms
Wall time: 928 ms


In [9]:
submit_part_b(second_answer)

aocd will not submit that answer again. At 2023-12-12 16:46:36.088424-05:00 you've previously submitted 8475948826693 and the server responded with:
That's the right answer!  You are one gold star closer to restoring snow operations.You have completed Day 12! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
